<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/title.png' width=500/></center>


# **이변량 분석 ③ - 범주형  → 범주형**

- 시각화와 수치화 방법으로 두 범주형 변수 간 관계를 분석합니다.

<img src='https://raw.githubusercontent.com/Jangrae/img/master/two_var_03.png' width=600 align='left'/>

<img src='https://raw.githubusercontent.com/Jangrae/img/master/analysis3.png' width=620 align='left'/>

## **1. 환경준비**

- 사용할 라이브러리와 분석 대상 데이터를 읽어옵니다.

In [ ]:
import numpy as np
import pandas as pd
import random as rd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.mosaicplot import mosaic
import scipy.stats as spst
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = (6, 4)

In [ ]:
# Titanic 데이터
path = 'https://raw.githubusercontent.com/Jangrae/csv/master/titanic.1.csv'
titanic = pd.read_csv(path)
titanic.head()

## **2. 교차표**

- 범주 vs 범주 를 비교하기 위해서는 Pandas의 **crosstab()** 함수를 사용해 **교차표**를 먼저 만들어야 합니다.
- **Sex → Survived** 관계를 비교하기 위해 교차표를 만들어봅니다.

In [ ]:
pd.crosstab(titanic['Sex'], titanic['Survived'])

- 참고로, 위 교차표는 다음과 같이 groupby() 메소드를 사용해 집계한 후 pivot 형태로 변환한 것과 같습니다.

In [ ]:
tmp = titanic.groupby(by=['Sex', 'Survived'], as_index=False)['Fare'].count()
tmp.columns = ['Sex', 'Survived', 'Cnt']
tmp.pivot(index='Sex', columns='Survived', values='Cnt')

- **Embarked → Survived** 관계를 비교하기 위해 교차표를 만들어봅니다.

In [ ]:
pd.crosstab(titanic['Embarked'], titanic['Survived'])

- 교차표는 다음과 같은 **normalize** 매개변수 값에 따라 다른 결과를 보여줍니다.
    - 'index': 각 행의 합으로 각 요소를 나눠서 각 행의 상대적인 비율을 보여줍니다(각 행의 합이 1이 됨).
    - 'columns': 각 열의 합으로 각 요소를 나눠서 각 열의 상대적인 비율을 보여줍니다(각 열이 합이 1이 됨).
    - 'all': 전체 합으로 각 요소를 나눠서 전체 데이터셋에서 각 셀의 상대적인 비율을 보여줍니다(전체 합이 1이 됨).

<img src='https://raw.githubusercontent.com/Jangrae/img/master/cross.png' width=700 align='left'/>


- **Embarked → Survived** 관계를 교차표로 normalize 매개변수 값을 바꿔가면서 표시해봅니다.

In [ ]:
pd.crosstab(titanic['Embarked'], titanic['Survived'], normalize='columns')

In [ ]:
pd.crosstab(titanic['Embarked'], titanic['Survived'], normalize='index')

In [ ]:
pd.crosstab(titanic['Embarked'], titanic['Survived'], normalize='all')

## **3. 시각화**

- 다음과 같은 방법으로 범주형 → 범주형 비교를 시각화할 수 있습니다.
    -  100% Stacked Bar
    -  Mosaic Plot


- 위 두 가지 방법으로 **Pclass → Survived** 관계를 시각화해서 비교해 봅니다.

**1) 100% Stacked Bar**

- 우선 **normalize='index'** 를 설정한 교차표를 작성합니다.
- Pclass 변수가 세 개의 범줏값을 가지므로 세 개의 행이 생깁니다.
- normalize='index'를 지정했으므로 각 행의 합은 1이 됩니다.

In [ ]:
table = pd.crosstab(titanic['Pclass'], titanic['Survived'], normalize='index')
table

- 위 교차표에 대해 Stacked Bar를 그립니다.
- Stacked Bar는 Pandas의 **plot()** 메소드로 쉽게 그릴 수 있습니다.
- 각 행별로 Stacked Bar가 그려지며, 각 행의 합이 1이므로 같은 높이를 갖습니다.
- 그래서 100% Stacked Bar 라고 부릅니다.
- **axhline()** 함수를 사용해 Survived 열 평균을 표시하는 가로선도 표시해봅니다.
- Survived 열 평균, 즉 titanic['Survived'].mean() 은 생존율을 의미합니다.
- 그런데 Stacked Bar 하단이 사망(Survived=0)을 의미합니다.
- 그래서 일관된 표시를 위해 1 - titanic['Survived'].mean() 을 표시합니다.

In [ ]:
table.plot(kind='bar', stacked=True)

plt.axhline(1-titanic['Survived'].mean(), color='r')
plt.xticks(rotation=0)
plt.show()

In [ ]:
table = pd.crosstab(titanic['Sex'], titanic['Survived'], normalize='index')
display(table)

table.plot(kind='bar', stacked=True)
plt.axhline(1-titanic['Survived'].mean(), color='r')
plt.xticks(rotation=0)
plt.show()

In [ ]:
table = pd.crosstab(titanic['Embarked'], titanic['Survived'], normalize='index')
display(table)

table.plot(kind='bar', stacked=True)
plt.axhline(1-titanic['Survived'].mean(), color='r')
plt.xticks(rotation=0)
plt.show()

<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_20.png' width=120 align="left"/>

titanic 데이터프레임의 다음 변수들 간의 관계를 100% Bar Plot 으로 시각화해 확인하세요.

**1️⃣ Title → Survived**

**2️⃣ AgeGroup → Survived**

<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_20e.png' width=45 align="left"/>

**2) Mosaic Plot**

- 100% Stacked Bar는 비율만 비교하므로 양에 대한 비교는 할 수 없습니다.
- Mosaic Plot으로 **양에 대한 비교**를 해봅니다.
- **tatsmodels.graphics.mosaicplot**의 **mosaic()** 함수로 Mosaic Plot을 그립니다.

In [ ]:
mosaic(titanic, ['Pclass', 'Survived'])
plt.axhline(1-titanic['Survived'].mean(), color='r')
plt.show()

- 위 그래프의 x축 길이는 각 객실 등급별 승객 비율을 나타냅니다.  
- y축 길이는 객실 승객의 사망, 생존 비율을 의미합니다.
- **sort_values()** 메소드를 사용해 원하는 순서로 정렬해 표시할 수 있습니다.
- **gap** 옵션으로 영역들 사이 간격을 조정할 수 있습니다.

In [ ]:
mosaic(titanic.sort_values(['Pclass', 'Survived']), ['Pclass', 'Survived'], gap=0.01)
plt.axhline(1-titanic['Survived'].mean(), color='r')
plt.show()

<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_20.png' width=120 align="left"/>

titanic 데이터프레임의 다음 변수들 간의 관계를 Mosaic Plot 으로 시각화해 확인하세요.

**1️⃣ Sex → Survived**

**2️⃣ Embarked → Survived**

**3️⃣ Title → Survived**

<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_20e.png' width=45 align="left"/>

**📌 시각화 결과를 분석하는 방법**

- 한 가지만 기억하세요! 귀무가설이 참일 때 나올 그래프의 모양(아래 그림)
- 여기서 벗어나면, '차이가 있다'., '관련이 있다'는 말입니다.



<img src='https://github.com/DA4BAM/image/blob/main/%EB%AA%A8%EC%9E%90%EC%9D%B5.png?raw=true' width=700/>

**참고:  Mosaic Plot을 3 범주로 그리기**

In [ ]:
mosaic(titanic, ['Sex', 'Pclass', 'Survived'], gap=0.02)
plt.show()

## **4. 수치화**

- 범주형 → 범주형 관계를 수치와해 비교할 때는 **카이제곱검정(Chi-Squared Test)** 을 사용합니다.
- **scipy.stats** 라이브러리의 **chi2_contingency()** 함수로 카이제곱검정을 수행합니다.
- 카이제곱검정은 $x^2$검정으로도 불립니다.

<img src = "https://github.com/Jangrae/img/blob/master/chi2.png?raw=true" width=300 align="left"/>

- 카이제곱통계량은
    - 클수록 기대빈도로부터 실제 값의 차이가 크다는 의미입니다.
    - 계산식으로 볼 때, 범주의 수가 늘어날 수록 값은 커지게 되어 있습니다.
    - 보통, 자유도의 2~3배 보다 크면, 차이가 있다고 봅니다.

- 범주형 변수의 자유도: 범주의 수 - 1

- 카이제곱검정에서는
    - x 변수의 자유도 × y 변수의 자유도
    - 예: Pclass → Survived
        - Pclass: 범주가 3개, Survived: 2개
        - (3-1) * (2-1) = 2
        - 그러므로, 2의 2 ~ 3배인 4 ~ 6 보다 카이제곱 통계량이 크면 차이가 있다고 볼수 있습니다.


- **Pclass → Survived** 관계를 카이제곱검정을 사용해 확인해봅니다.
- 우선 교차표를 만들고 확인합니다.

In [ ]:
table = pd.crosstab(titanic['Pclass'], titanic['Survived'])
table

- 이 교차표를 사용해 카이제곱검정을 수행합니다.

In [ ]:
result = spst.chi2_contingency(table)
print('* 카이제곱통계량:', result[0])
print('* p-value:', result[1])
print('* 기대빈도:\n',result[3])

<img src = "https://github.com/Jangrae/img/blob/master/chi.png?raw=true" width=800 align="left"/>

📌 **기대빈도 계산**

**1)** $\frac{549 \times 216}{891}$
**2)** $\frac{549 \times 184}{891}$
**3)** $\frac{549 \times 491}{891}$
**4)** $\frac{342 \times 216}{891}$
**5)** $\frac{342 \times 184}{891}$
**6)** $\frac{342 \times 491}{891}$



📌 **카이제곱통계량 계산**

$\large x^2$$=\frac{(80-133.0909)^2}{133.0909}+\frac{(97-113.3737)^2}{113.3737}+\frac{(372-302.5354)^2}{302.5354}+\frac{(136-82.90909)^2}{82.90909}+\frac{(87-70.62626)^2}{70.62626}+\frac{(119-188.4646)^2}{188.4646}=102.889$

<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_20.png' width=120 align="left"/>

titanic 데이터프레임의 다음 변수들 간의 관계를 카이제곱검정으로 확인하세요.

**1️⃣ Sex → Survived**

**2️⃣ Embarked → Survived**

**3️⃣ Title → Survived**

<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_20e.png' width=45 align="left"/>